In [1]:
from config import *
from utilities import get_model_name
from dataset import ClassificationDataset

import functools
import numpy as np
import pandas as pd

results_dict = {}
internal, external = 11, 12
encode_method = "dummy"
impute_method = "itlsvm" #TODO
fs_method, fs_ratio = "dnp", 0.5  #dnp & graces
norm_method = "maxmin"
classification_dataset = ClassificationDataset(
    internal, external, encode_method, impute_method, fs_method, fs_ratio, norm_method, random_state=SEED)

Number of Selected Features: 100%|██████████| 71/71 [06:12<00:00,  5.24s/it]


In [2]:
# from sklearn.neural_network import MLPClassifier
from utilities import MLPClassifier

model_fn = functools.partial(
    MLPClassifier, random_state=SEED, max_iter=50,
    momentum=0.9, squares=0.999, optimizer_t='sgdm',
    use_residual=True, use_batch_norm=False)
param_grid_list = [{
    # hidden_layer_sizes = (64, 32)
    # config       lr_range            worst_auc
    # adam w/ bn:  [2e-5, 5e-5, 1e-4]  0.8942
    # adam w/o bn: [5e-5, 1e-4, 2e-4]  0.9148
    # sgdm w/ bn:  [1e-3, 2e-3, 5e-3]  0.8982
    # sgdm w/o bn: [5e-3, 1e-2, 2e-2]  0.9177

    'hidden_layer_sizes': [(128, 128), (64, 64, 64)],
    'learning_rate': [5e-3, 1e-2, 2e-2],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

MLPClassifier


Epoch 1/50:  54%|█████▍    | 14/26 [00:00<00:00, 136.49batch/s, loss=0.284]

model(hidden_layer_sizes=(128, 128), learning_rate=0.005)
Valid:         AUC: 0.9147 | AUPR: 0.6502 | Acc: 95.35%


model(hidden_layer_sizes=(128, 128), learning_rate=0.01)
Valid:         AUC: 0.9183 | AUPR: 0.6588 | Acc: 95.33%


model(hidden_layer_sizes=(128, 128), learning_rate=0.02)
Valid:         AUC: 0.9210 | AUPR: 0.6665 | Acc: 95.35%


model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.005)
Valid:         AUC: 0.9162 | AUPR: 0.6514 | Acc: 95.39%


model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.01)
Valid:         AUC: 0.9194 | AUPR: 0.6608 | Acc: 95.33%


model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.02)
Valid:         AUC: 0.9209 | AUPR: 0.6675 | Acc: 95.38%
best model(hidden_layer_sizes=(128, 128), learning_rate=0.02)
Valid:         AUC: 0.9210 | AUPR: 0.6665 | Acc: 95.35%
Internal test: AUC: 0.9285 | AUPR: 0.6964 | Acc: 95.45%
External test: AUC: 0.9204 | AUPR: 0.6608 | Acc: 95.01%


In [3]:
from sklearn.svm import SVC

model_fn = functools.partial(
    SVC, random_state=SEED, probability=True,
    class_weight='balanced')
param_grid_list = [{
    'kernel': ['linear'],
    'C': [0.01, 0.1, 1],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

SVC
model(kernel=linear, C=0.01)
Valid:         AUC: 0.9157 | AUPR: 0.6180 | Acc: 94.98%
model(kernel=linear, C=0.1)
Valid:         AUC: 0.9155 | AUPR: 0.6127 | Acc: 94.98%
model(kernel=linear, C=1)
Valid:         AUC: 0.9140 | AUPR: 0.6055 | Acc: 94.92%
best model(kernel=linear, C=0.01)
Valid:         AUC: 0.9157 | AUPR: 0.6180 | Acc: 94.98%
Internal test: AUC: 0.9223 | AUPR: 0.6456 | Acc: 94.82%
External test: AUC: 0.9153 | AUPR: 0.5963 | Acc: 94.52%


In [4]:
from sklearn.neighbors import KNeighborsClassifier

model_fn = functools.partial(
    KNeighborsClassifier, n_jobs=-1, algorithm='auto', p=2)
param_grid_list = [{
    'n_neighbors': [3, 5, 7],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

KNeighborsClassifier
model(n_neighbors=3)
Valid:         AUC: 0.7607 | AUPR: 0.4197 | Acc: 94.53%
model(n_neighbors=5)
Valid:         AUC: 0.7869 | AUPR: 0.4660 | Acc: 94.88%
model(n_neighbors=7)
Valid:         AUC: 0.8016 | AUPR: 0.4932 | Acc: 94.94%
best model(n_neighbors=7)
Valid:         AUC: 0.8016 | AUPR: 0.4932 | Acc: 94.94%
Internal test: AUC: 0.8011 | AUPR: 0.5062 | Acc: 94.72%
External test: AUC: 0.8097 | AUPR: 0.4979 | Acc: 94.84%


In [5]:
from sklearn.naive_bayes import GaussianNB

model_fn = GaussianNB
param_grid_list = [{}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

GaussianNB
model()
Valid:         AUC: 0.8758 | AUPR: 0.4462 | Acc: 83.37%
best model()
Valid:         AUC: 0.8758 | AUPR: 0.4462 | Acc: 83.37%
Internal test: AUC: 0.8908 | AUPR: 0.4776 | Acc: 84.14%
External test: AUC: 0.8766 | AUPR: 0.4294 | Acc: 82.27%


In [6]:
# from sklearn.exceptions import ConvergenceWarning
# import warnings
# warnings.filterwarnings("ignore", category=ConvergenceWarning)
from sklearn.linear_model import LogisticRegression

model_fn = functools.partial(
    LogisticRegression, random_state=SEED, n_jobs=-1, 
    max_iter=2000, solver='saga', penalty='l2',
    )
param_grid_list = [{
    'C': [1.0, 10.0, 100.0],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(
    model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(
    best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

LogisticRegression


model(C=1.0)
Valid:         AUC: 0.9159 | AUPR: 0.6486 | Acc: 95.22%
model(C=10.0)
Valid:         AUC: 0.9152 | AUPR: 0.6473 | Acc: 95.24%
model(C=100.0)
Valid:         AUC: 0.9145 | AUPR: 0.6466 | Acc: 95.27%
best model(C=1.0)
Valid:         AUC: 0.9159 | AUPR: 0.6486 | Acc: 95.22%
Internal test: AUC: 0.9231 | AUPR: 0.6768 | Acc: 95.06%
External test: AUC: 0.9126 | AUPR: 0.6263 | Acc: 94.92%


In [7]:
from sklearn.tree import DecisionTreeClassifier

model_fn = functools.partial(
    DecisionTreeClassifier, random_state=SEED,
    ccp_alpha=0.0, class_weight='balanced', max_depth=5)
param_grid_list = [{
        'criterion': ['gini', 'log_loss', 'entropy'],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

DecisionTreeClassifier
model(criterion=gini)
Valid:         AUC: 0.8658 | AUPR: 0.5430 | Acc: 83.47%
model(criterion=log_loss)
Valid:         AUC: 0.8673 | AUPR: 0.5619 | Acc: 81.57%
model(criterion=entropy)
Valid:         AUC: 0.8673 | AUPR: 0.5619 | Acc: 81.57%
best model(criterion=log_loss)
Valid:         AUC: 0.8673 | AUPR: 0.5619 | Acc: 81.57%
Internal test: AUC: 0.8796 | AUPR: 0.6046 | Acc: 81.70%
External test: AUC: 0.8728 | AUPR: 0.5818 | Acc: 81.24%


In [8]:
from sklearn.ensemble import RandomForestClassifier

model_fn = functools.partial(
    RandomForestClassifier, random_state=SEED, n_jobs=-1,
    ccp_alpha=0.0, class_weight='balanced', max_depth=5, bootstrap=False)
param_grid_list = [{
        'criterion': ['gini', 'log_loss', 'entropy'],
        'n_estimators': [50, 100, 200],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

RandomForestClassifier
model(criterion=gini, n_estimators=50)
Valid:         AUC: 0.9136 | AUPR: 0.6506 | Acc: 89.92%
model(criterion=gini, n_estimators=100)
Valid:         AUC: 0.9149 | AUPR: 0.6537 | Acc: 90.00%
model(criterion=gini, n_estimators=200)
Valid:         AUC: 0.9147 | AUPR: 0.6540 | Acc: 89.96%
model(criterion=log_loss, n_estimators=50)
Valid:         AUC: 0.9128 | AUPR: 0.6535 | Acc: 90.26%
model(criterion=log_loss, n_estimators=100)
Valid:         AUC: 0.9138 | AUPR: 0.6550 | Acc: 90.15%
model(criterion=log_loss, n_estimators=200)
Valid:         AUC: 0.9139 | AUPR: 0.6545 | Acc: 90.22%
model(criterion=entropy, n_estimators=50)
Valid:         AUC: 0.9128 | AUPR: 0.6535 | Acc: 90.26%
model(criterion=entropy, n_estimators=100)
Valid:         AUC: 0.9138 | AUPR: 0.6550 | Acc: 90.15%
model(criterion=entropy, n_estimators=200)
Valid:         AUC: 0.9139 | AUPR: 0.6545 | Acc: 90.22%
best model(criterion=gini, n_estimators=100)
Valid:         AUC: 0.9149 | AUPR: 0.6537 | Acc: 9

In [9]:
from sklearn.ensemble import GradientBoostingClassifier

model = functools.partial(
    GradientBoostingClassifier, random_state=SEED,
    learning_rate=0.1, max_depth=5, loss='log_loss', 
    n_estimators=100)
param_grid_list = [{
        'criterion': ['friedman_mse', 'squared_error'],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

GradientBoostingClassifier
model(criterion=friedman_mse)
Valid:         AUC: 0.9174 | AUPR: 0.6550 | Acc: 95.20%
model(criterion=squared_error)
Valid:         AUC: 0.9173 | AUPR: 0.6551 | Acc: 95.20%
best model(criterion=friedman_mse)
Valid:         AUC: 0.9174 | AUPR: 0.6550 | Acc: 95.20%
Internal test: AUC: 0.9252 | AUPR: 0.6855 | Acc: 95.11%
External test: AUC: 0.9216 | AUPR: 0.6467 | Acc: 95.10%


In [10]:
from sklearn.ensemble import AdaBoostClassifier
model = functools.partial(AdaBoostClassifier, random_state=SEED, algorithm="SAMME")
param_grid_list = [{
    'n_estimators': [100, 200, 500],
    'learning_rate': [1.0],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

AdaBoostClassifier
model(n_estimators=100, learning_rate=1.0)
Valid:         AUC: 0.9125 | AUPR: 0.6354 | Acc: 95.05%
model(n_estimators=200, learning_rate=1.0)
Valid:         AUC: 0.9141 | AUPR: 0.6362 | Acc: 95.01%
model(n_estimators=500, learning_rate=1.0)
Valid:         AUC: 0.9135 | AUPR: 0.6359 | Acc: 95.08%
best model(n_estimators=200, learning_rate=1.0)
Valid:         AUC: 0.9141 | AUPR: 0.6362 | Acc: 95.01%
Internal test: AUC: 0.9175 | AUPR: 0.6730 | Acc: 95.01%
External test: AUC: 0.9144 | AUPR: 0.6237 | Acc: 94.75%


In [11]:
dataframe = pd.DataFrame(results_dict).transpose().reset_index()
dataframe.columns = ['model', 'val_auc', 'val_aupr', 'val_acc', 'intest_auc', 'intest_aupr', 'intest_acc', 'extest_auc', 'extest_aupr', 'extest_acc']
# dataframe.to_csv()
filename = os.path.join(RESULTS_DIR, encode_method, impute_method, f'{fs_method}{int(fs_ratio * 142)}.csv')
dirname = os.path.dirname(filename)
if not os.path.exists(dirname):
    os.makedirs(dirname)
dataframe.to_csv(filename, index=False)